## Get judges from case

Get cases json from the API, return all judges from that case

Map the judges to a CourtListener URL

In [1]:
import re
import os
import sys
sys.path.append('..')

import requests

from config import settings
import utils

In [2]:
url = utils.get_api_url() + 'cases/?jurisdiction=ill&full_case=true&format=json'
headers = {'AUTHORIZATION': 'Token {}'.format(settings.API_KEY)}
response = requests.get(url, headers=headers)
res = response.json()

In [3]:
cases = res['results']
print(len(cases), "results found")

100 results found


In [4]:
not_useful = ["chief", "justice", "judge", "and", "associate","mr", "mrs", "ms", "presiding","opinion"]
not_names = ["memorandum", "per curiam", "opinion of the court", "the court", "dissenting", "justice" ]
courtlistener_people_api = "https://www.courtlistener.com/api/rest/v3/people/"
def get_author_name(idx, opinion):
    judges = []
    try:
        authors = opinion['author'].split(', ')
        for author in authors:
            original_author = author
            author_name = fix_name(author)
            if author_name in not_names:
                continue
            first_name = ''
            name_parts = author_name.split(' ')
               
            if len(name_parts) > 1:
                author_name = name_parts[-1]
                if len(name_parts[0]) > 1:
                    first_name = name_parts[0]
            if len(author_name) > 1 and author_name not in not_names:
                author_name = author_name.capitalize() 
                if '-' in author_name:
                    # keep hyphenations in people's names, capitalize each hyphenated part
                    name_parts = author_name.split('-')
                    author_name = ''
                    for idx,part in enumerate(name_parts):
                        if idx < len(name_parts) - 1:
                            author_name += part.capitalize() + '-' 
                        else:
                            author_name += part.capitalize()
                url = "%s?name_last=%s" % (courtlistener_people_api, author_name)
                if first_name:
                    url += "&name_first=%s" % first_name.capitalize()
                judges.append([idx, opinion['type'], url, original_author])
    except:
        # for one reason or another the opinion doesn't have an author. 
        # Sometimes it's because the judges have decided that case is not to be treated as precedential
        pass
    return judges

def fix_name(name):
    name = re.sub('[^0-9a-zA-Z\-\s]+', '', name)
    name = name.lower()
    for nu in not_useful:
        name = name.replace(nu + ' ','')
    return name.strip()


def get_all_judges(cases, opinion_type=None):
    judge_results = []
    for case in cases:

        opinions = case['casebody']['data']['opinions']

        for opinion in opinions:
            if opinion_type:
                if opinion['type'] == opinion_type:
                    names = get_author_name(case['id'],opinion)
            else:
                names = get_author_name(case['id'],opinion)
            if names:
                judge_results += names
    return judge_results
judges = get_all_judges(cases)

# print first five
print(judges[0:5])

[[2747110, 'majority', 'https://www.courtlistener.com/api/rest/v3/people/?name_last=Eberspacher', 'Mr. PRESIDING JUSTICE EBERSPACHER'], [435530, 'majority', 'https://www.courtlistener.com/api/rest/v3/people/?name_last=Reynolds&name_first=John', 'Justice John Reynolds.'], [435536, 'majority', 'https://www.courtlistener.com/api/rest/v3/people/?name_last=Reynolds', 'Chief Justice Reynolds,'], [435546, 'majority', 'https://www.courtlistener.com/api/rest/v3/people/?name_last=Reynolds&name_first=John', 'Justice John Reynolds.'], [435548, 'majority', 'https://www.courtlistener.com/api/rest/v3/people/?name_last=Reynolds', 'Chief Justice Reynolds.']]


## DataFrames!

In [5]:
import pandas as pd
judge_df = pd.DataFrame(data=judges, columns=["id","type","best_guess_url","original"])
judge_df.head()

,id,type,best_guess_url,original
0,2747110,majority,https://www.courtlistener.com/api/rest/v3/peop...,Mr. PRESIDING JUSTICE EBERSPACHER
1,435530,majority,https://www.courtlistener.com/api/rest/v3/peop...,Justice John Reynolds.
2,435536,majority,https://www.courtlistener.com/api/rest/v3/peop...,"Chief Justice Reynolds,"
3,435546,majority,https://www.courtlistener.com/api/rest/v3/peop...,Justice John Reynolds.
4,435548,majority,https://www.courtlistener.com/api/rest/v3/peop...,Chief Justice Reynolds.
